<a href="https://colab.research.google.com/github/udaymallappa/learning-to-route/blob/master/Sim2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recursive https://github.com/udaymallappa/learning-to-route.git

Cloning into 'learning-to-route'...
remote: Enumerating objects: 105, done.
remote: Total 105 (delta 0), reused 0 (delta 0), pack-reused 105
Receiving objects: 100% (105/105), 1.60 MiB | 3.54 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [0]:
import os, sys
sys.path.append('..')
import numpy as np
import torch
import joblib
from datetime import datetime

In [3]:
!env CUDA_VISIBLE_DEVICES=1
print("Numpy: {}, Torch: {}, Joblib: {}".format(np.__version__, torch.__version__, joblib.__version__))

CUDNN_VERSION=7.6.5.32
__EGL_VENDOR_LIBRARY_DIRS=/usr/lib64-nvidia:/usr/share/glvnd/egl_vendor.d/
LD_LIBRARY_PATH=/usr/lib64-nvidia
_=/usr/bin/env
LANG=en_US.UTF-8
HOSTNAME=45d82ab359bf
OLDPWD=/
CLOUDSDK_CONFIG=/content/.config
KMP_INIT_AT_FORK=FALSE
NVIDIA_VISIBLE_DEVICES=all
DATALAB_SETTINGS_OVERRIDES={"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\"172.28.0.2\""]}
ENV=/root/.bashrc
PAGER=cat
NCCL_VERSION=2.4.8
TF_FORCE_GPU_ALLOW_GROWTH=true
JPY_PARENT_PID=26
NO_GCE_CHECK=True
PWD=/content
HOME=/root
LAST_FORCED_REBUILD=20200204
CLICOLOR=1
DEBIAN_FRONTEND=noninteractive
LIBRARY_PATH=/usr/local/cuda/lib64/stubs
GLIBCPP_FORCE_NEW=1
TBE_CREDS_ADDR=172.28.0.1:8008
TERM=xterm-color
SHELL=/bin/bash
GCS_READ_CACHE_BLOCK_SIZE_MB=16
PYTHONWARNINGS=ignore:::pip._internal.cli.base_command
MPLBACKEND=module://ipykernel.pylab.backend_inline
CUDA_PKG_VERSION=10-1=10.1.243-1
CUDA_VERSION=10.1.243
NVIDIA_DRIVER_CAPABILITIES=compute,utility
SHLVL=2
PYTHONPAT

In [4]:
DATA_DIR = './data/DEEP100K'

if not os.path.exists(DATA_DIR):
    assert not DATA_DIR.endswith(os.sep), 'please do not put "/" at the end of DATA_DIR'
    !mkdir -p {DATA_DIR}
    !wget https://www.dropbox.com/sh/oc8tswkzrdqe4vi/AADSOL74QIBtg_EvOdBYOX3na?dl=1 -O {DATA_DIR}/deep_100k.zip
    !cd {DATA_DIR} && unzip deep_100k.zip

--2020-02-11 06:39:53--  https://www.dropbox.com/sh/oc8tswkzrdqe4vi/AADSOL74QIBtg_EvOdBYOX3na?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/oc8tswkzrdqe4vi/AADSOL74QIBtg_EvOdBYOX3na [following]
--2020-02-11 06:39:53--  https://www.dropbox.com/sh/dl/oc8tswkzrdqe4vi/AADSOL74QIBtg_EvOdBYOX3na
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc69136df73a7a2318be62ec0753.dl.dropboxusercontent.com/zip_download_get/AVS98_AyjlmrzSHFz40l1Jlw975gzEfts3rgB9fcXjQ0QPq_e7H52Q1F2GFRabJX9UehYI-QFTQg6d03RITi9eW52NkCKjZ9nW4RXUZiBC-gXQ?dl=1 [following]
--2020-02-11 06:39:54--  https://uc69136df73a7a2318be62ec0753.dl.dropboxusercontent.com/zip_download_get/AVS98_AyjlmrzSHFz40l1Jlw975gzEfts3rgB9fcXjQ0QPq_e7H52Q1F2GFRabJX9UehYI-QFTQg6d

In [0]:
import os.path as osp

graph_params = { 
    'vertices_path': osp.join(DATA_DIR, 'deep_base.fvecs'),
    'edges_path': osp.join(DATA_DIR, 'test_hnsw100k_M8_ef300_onelevel1.ivecs'),
    'initial_vertex_id': 0,         # by default, starts search from this vertex
#     'edges_path': osp.join(DATA_DIR, 'test_hnsw100k_M8_ef300_onelevel0.ivecs'),
#     'info_path': osp.join(DATA_DIR, 'test_hnsw100k_M8_ef300_onelevel0.bin'),
#     ^-- uncomment this to get hierarchical NSW
    
    'train_queries_path': osp.join(DATA_DIR, 'deep_learn.fvecs'),
    'test_queries_path': osp.join(DATA_DIR, 'deep_query.fvecs'),
    
    'train_gt_path': osp.join(DATA_DIR, 'train_gt.ivecs'),
    'test_gt_path': osp.join(DATA_DIR, 'test_gt.ivecs'),
#     ^-- comment these 2 lines to re-compute ground truth ids (if you don't have pre-computed ground truths)

    'ground_truth_n_neighbors': 1,  # for each query, finds this many nearest neighbors via brute force
}



In [6]:
!pwd

/content


In [7]:
%cd learning-to-route/

/content/learning-to-route


In [8]:
!pip install tensorboardX

     |████████████████████████████████| 204kB 8.7MB/s 


In [9]:
!pwd

/content/learning-to-route


In [10]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (8,895 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fr

In [11]:
!pwd

/content/learning-to-route


In [0]:
from lib1.utils import global_variables

In [14]:
import lib1

running clean
running build
running build_ext
building '_bfs' extension
swigging bfs.i to bfs_wrap.c
swig -python -threads -o bfs_wrap.c bfs.i
creating build
creating build/temp.linux-x86_64-3.6
g++ -std=c++11 -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I/usr/include/python3.6m -c bfs_wrap.c -o build/temp.linux-x86_64-3.6/bfs_wrap.o -fopenmp
g++ -std=c++11 -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I/usr/include/python3.6m -c bfs.cc -o build/temp.linux-x86_64-3.6/bfs.o -fopenmp
creating build/lib.linux-x86_64-3.6
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Wer

In [18]:
%cd ..

/content


In [0]:
from lib1.utils import global_variables

In [0]:
from lib1.graph import Graph

In [0]:
graph = Graph(**graph_params)

In [0]:

total_dcs = 128

routing_dcs = 120
routing_vector_size = 96
verification_top_k = 8

architecture = 'gcn'      # 'gcn' or 'ffn' or whatever you define manually
max_batches  = 60000       # num training iterations; use float('inf') for unlimited

# one cycle learning rate schedule
learning_rate_base = 1e-3
warmup_steps = 5000
decay_rate = 1./3
learning_rate_min = 1e-5
batch_size = 1024
dev_batch_size = 100

# batch generator
sampling_temperature = 1.0
sampling_chunk_size = 10000
n_jobs = 8
margin = 5

In [23]:
exp_name = '{data_name}_dcs{total_dcs}_{architecture}_size{routing_vector_size}_routing{routing_dcs}_verification{verification_top_k}'.format(
    data_name=osp.split(DATA_DIR)[-1], total_dcs=total_dcs, routing_vector_size=routing_vector_size, routing_dcs=routing_dcs,
    verification_top_k=verification_top_k, architecture=architecture
)
print('exp name:', exp_name)
!rm {'./runs/' + exp_name} -rf # KEEP COMMENTED!
assert not os.path.exists('./runs/' + exp_name)

original_vector_size = graph.vertices.shape[-1]
is_compressed = (routing_vector_size != graph.vertices.shape[1])
if not is_compressed:
    projection_cost = 0
else:
    projection_cost = routing_vector_size 
sanity_check_dcs = round(routing_dcs * routing_vector_size / original_vector_size + projection_cost + verification_top_k)
assert total_dcs == sanity_check_dcs, "current scheme has {} dcs out of {} required".format(sanity_check_dcs, total_dcs)

exp name: DEEP100K_dcs128_gcn_size96_routing120_verification8


In [24]:
import lib1
dir(lib1)
!pwd
from lib1 import hnsw

/content


In [0]:
hnsw = hnsw.WalkerHNSW(graph, max_dcs=routing_dcs,
                           top_vertices_for_verification=verification_top_k,
                           distance_for_verification=hnsw.DISTANCES['euclidian'],
                           distance_for_routing=hnsw.DISTANCES['negative_dot'],
                           hierarchical=False
)

In [0]:

from tensorboardX import SummaryWriter
from lib1 import walker_agent
if architecture == 'gcn':
    agent = walker_agent.GCNWalkerAgent(
        graph.vertices.shape[1], conv_hid_size=256, hidden_size=1024, output_size=routing_vector_size, 
        residual=not is_compressed, project_query=is_compressed).to(device='cuda')

elif architecture == 'ffn':
    agent = walker_agent.SimpleWalkerAgent(
        graph.vertices.shape[1], hidden_size=4096, output_size=routing_vector_size, 
        residual=not is_compressed, project_query=is_compressed).to(device='cuda')

else:
    raise ValueError("Unknown architecture: {}".format(architecture))



In [27]:
from lib1 import ParallelBFS
oracle = ParallelBFS(graph_edges=graph.edges)

!pwd

/content


In [0]:
from lib1 import SupervisedWalkerTrainer
trainer = SupervisedWalkerTrainer(
    agent, hnsw, oracle, writer=SummaryWriter('runs/' + exp_name),
    learning_rate_base=learning_rate_base, warmup_steps=warmup_steps,
    decay_rate=decay_rate, learning_rate_min=learning_rate_min,
)

In [29]:
trainer.step

0

In [0]:
from pandas import DataFrame
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
%matplotlib inline
moving_average = lambda x, **kw: DataFrame({'x':np.asarray(x)}).x.ewm(**kw).mean().values
train_loss_history, train_acc_history, dev_recall_history, total_dev_recall_history = [], [], [], []

In [0]:
# generates batches of [queries, ground truth]
from lib1 import utils
dev_batcher = utils.iterate_minibatches(graph.test_queries, graph.test_gt, batch_size=dev_batch_size)

# generates batches of [records] for training
train_batcher = trainer.generate_training_batches(
    graph.train_queries, graph.train_gt,
    queries_per_chunk=sampling_chunk_size, batch_size_max=batch_size,
    sampling_temperature=sampling_temperature, n_jobs=n_jobs)

In [43]:
%cd learning-to-route/

/content/learning-to-route


In [63]:
!ls -a
!git config --global user.email "udaymallappa@gmail.com"
!git config --global user.name "Uday Mallappa"
!git init
!git add .
!git commit -m "fixed lib name issues"

.   .git	images	     .ipynb_checkpoints  notebooks  requirements.txt
..  .gitignore	__init__.py  lib1		 README.md
Reinitialized existing Git repository in /content/learning-to-route/.git/
On branch online
Your branch is ahead of 'origin/online' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
!git add .

In [75]:
!git commit -m "update2"

[online 92f8dd1] update2
 17 files changed, 0 insertions(+), 0 deletions(-)
 rename {lib1 => lib}/__init__.py (100%)
 rename {lib1 => lib}/bfs_swig/__init__.py (100%)
 rename {lib1 => lib}/bfs_swig/bfs.cc (100%)
 rename {lib1 => lib}/bfs_swig/bfs.h (100%)
 rename {lib1 => lib}/bfs_swig/bfs.i (100%)
 rename {lib1 => lib}/bfs_swig/bfs.py (100%)
 rename {lib1 => lib}/bfs_swig/bfs_wrap.c (100%)
 rename {lib1 => lib}/bfs_swig/numpy.i (100%)
 rename {lib1 => lib}/bfs_swig/setup.py (100%)
 rename {lib1 => lib}/bfs_swig/test_bfs.py (100%)
 rename {lib1 => lib}/bfs_swig/wrapper.py (100%)
 rename {lib1 => lib}/graph.py (100%)
 rename {lib1 => lib}/hnsw.py (100%)
 rename {lib1 => lib}/nn_utils.py (100%)
 rename {lib1 => lib}/utils.py (100%)
 rename {lib1 => lib}/walker_agent.py (100%)
 rename {lib1 => lib}/walker_trainer.py (100%)


In [77]:
!git push -f origin master

error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/udaymallappa/learning-to-route.git'


In [62]:
!git status

On branch online
Your branch is ahead of 'origin/online' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [78]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [82]:
%cd ..

/content


In [86]:
!pwd
!ls

/content
data  drive  learning-to-route	runs  sample_data


In [90]:
%cd /content/drive/My Drive/Tree

/content/drive/My Drive/Tree


In [91]:
!pwd

/content/drive/My Drive/Tree


In [99]:
%cd /content/drive/My Drive/Tree

/content/drive/My Drive/Tree


In [100]:
!ls -a 

bfs.cc	graph.py	   numpy.i	temp		 walker_trainer.py
bfs.h	hnsw.py		   __pycache__	test_bfs.py	 wrapper.py
bfs.i	learning-to-route  search.py	utils.py
data	nn_utils.py	   setup.py	walker_agent.py


In [101]:
!echo "# graph_route" >> README.md
!git init

Initialized empty Git repository in /content/drive/My Drive/Tree/.git/


In [111]:
!pwd
!git init
!ls -a

/content/drive/My Drive/Tree
Initialized empty Git repository in /content/drive/My Drive/Tree/.git/
bfs.cc	.git		   nn_utils.py	search.py    utils.py
bfs.h	graph.py	   numpy.i	setup.py     walker_agent.py
bfs.i	hnsw.py		   __pycache__	temp	     walker_trainer.py
data	learning-to-route  README.md	test_bfs.py  wrapper.py


In [113]:
! git add .

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> learning-to-route
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached learning-to-route
hint: 
hint: See "git help submodule" for more information.


In [114]:
!git commit -m "first commit"

[master (root-commit) feb7c98] first commit
 49 files changed, 18653 insertions(+)
 create mode 100644 README.md
 create mode 100644 __pycache__/graph.cpython-36.pyc
 create mode 100644 __pycache__/hnsw.cpython-36.pyc
 create mode 100644 __pycache__/nn_utils.cpython-36.pyc
 create mode 100644 __pycache__/search.cpython-36.pyc
 create mode 100644 __pycache__/utils.cpython-36.pyc
 create mode 100644 __pycache__/walker_agent.cpython-36.pyc
 create mode 100644 __pycache__/wrapper.cpython-36.pyc
 create mode 100644 bfs.cc
 create mode 100644 bfs.h
 create mode 100644 bfs.i
 create mode 100644 data/DEEP100K/deep_100k.zip
 create mode 100644 data/DEEP100K/deep_base.fvecs
 create mode 100644 data/DEEP100K/deep_base_pca24.fvecs
 create mode 100644 data/DEEP100K/deep_base_pca48.fvecs
 create mode 100644 data/DEEP100K/deep_learn.fvecs
 create mode 100644 data/DEEP100K/deep_learn_pca24.fvecs
 create mode 100644 data/DEEP100K/deep_learn_pca48.fvecs
 create mode 100644 data/DEEP100K/deep_query.fvecs

In [124]:
!ls -a 

bfs.cc	.git		   nn_utils.py	search.py    utils.py
bfs.h	graph.py	   numpy.i	setup.py     walker_agent.py
bfs.i	hnsw.py		   __pycache__	temp	     walker_trainer.py
data	learning-to-route  README.md	test_bfs.py  wrapper.py


In [0]:
!git remote rm origin

In [0]:
!git config --global user.name "Uday Mallappa"
!git config --global user.email udaymallappa@gmail.com

In [0]:
!git remote set-url origin git@github.com:udaymallappa/graph_route.git

fatal: could not read Username for 'https://github.com': No such device or address


In [37]:
!pwd
max_batches
trainer.step
for batch_records in train_batcher:
  print('hi')


/content
Precomputing initial vertices... Done!

Generating new batch of trajectories... 

NameError: ignored

In [0]:
for batch_records in train_batcher:
    print('Hello')
    metrics_t = trainer.train_on_batch(batch_records)
    train_loss_history.append(metrics_t['loss'].item())
    train_acc_history.append(metrics_t['acc'].item())
    print(end='.')
    print(trainer.step)
    if trainer.step % 100 == 0:
        dev_metrics_t = trainer.compute_dev_metrics_on_batch(*next(dev_batcher))
        dev_recall_history.append(dev_metrics_t['recall'])
    
    if trainer.step % 5000 == 0:
        print(end="Saving...")
        torch.save(agent, "runs/{}/agent.{}.pth".format(exp_name, trainer.step))
        torch.save(trainer.opt, "runs/{}/opt.{}.pth".format(exp_name, trainer.step))
        print('Done!')
        print('Compute metrics for all test queries...')
        total_dev_metrics_t = trainer.compute_dev_metrics_on_batch(graph.test_queries, graph.test_gt, prefix='/total_dev')
        total_dev_recall_history.append(total_dev_metrics_t['recall'])
        print('Done')
        
    if trainer.step % 100 == 0:
        clear_output(True)
        plt.figure(figsize=[18, 6])
        plt.subplot(1, 3, 1)
        plt.title('train loss over time'); plt.grid();
        plt.plot(moving_average(train_loss_history, span=50))
        plt.scatter(range(len(train_loss_history)), train_loss_history, alpha=0.1)
        plt.subplot(1, 3, 2)
        plt.title('train acc over time'); plt.grid();
        plt.plot(moving_average(train_acc_history, span=50))
        plt.scatter(range(len(train_acc_history)), train_acc_history, alpha=0.1)
        plt.subplot(1, 3, 3)
        plt.title('dev recall over time'); plt.grid();
        plt.plot(moving_average(dev_recall_history, span=10))
        plt.scatter(range(len(dev_recall_history)), dev_recall_history, alpha=0.1)
        plt.plot(50*(np.arange(len(total_dev_recall_history)) + 1), total_dev_recall_history, c='r')
        plt.show()
        
        print("step=%i, mean_loss=%.3f, time=%.3f" % 
              (len(train_loss_history), np.mean(train_loss_history[-100:]), metrics_t['step_time']))
        print('_' * 100)
    
    if trainer.step > max_batches: break


In [0]:
def compute_recalls(queries, gt, hnsw=hnsw, agent=agent, wrap=lambda x:x, **kwargs):
    """ Compute recall, mean distance computations and mean number of hops """
    with torch.no_grad():
        state = agent.prepare_state(graph, device='cuda')
        predictions = [hnsw.find_nearest(q, state=state, agent=agent) for q in wrap(queries)]
        
    answers = [prediction['best_vertex_id'] for prediction in predictions]
    mean_recall = np.mean([best in gt for best, gt in zip(answers, gt)])
    distances = [prediction['routing_dcs'] for prediction in predictions]
    hops = [prediction['num_hops'] for prediction in predictions]
    return dict(recall=mean_recall, dcs=np.mean(distances), hops=np.mean(hops))